Five steps:


    1. Designing model (input, output size, forward pass)
    2. construct loss and optmizer
    3. Training loop:
        3.a. forward pass: compute prediction and loss
        3.b. backward pass: gridients
        3.c. update weights

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [10]:
# prepare the data

bc = datasets.load_breast_cancer()

x,y = bc.data, bc.target

n_samples, n_features = x.shape
print(n_samples, n_features)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1234)

#scale

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))


y_train = y_train.view(y_train.shape[0], 1) #converting row vector column vector
y_test = y_test.view(y_test.shape[0], 1)
# model

class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))

        return y_pred


model = LogisticRegression(n_features)


# loss and optmizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
# training loops

n_epochs = 100

for epoch in range(n_epochs):
    y_pred = model(x_train)

    loss = criterion(y_pred, y_train)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.8f}')

with torch.no_grad():
    y_pred = model(x_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() /float(y_test.shape[0])
    print(f'Accuracy = {acc*100:.4f}%')
 



569 30
epoch: 10, loss = 0.48005968
epoch: 20, loss = 0.41683772
epoch: 30, loss = 0.37265998
epoch: 40, loss = 0.33986184
epoch: 50, loss = 0.31439954
epoch: 60, loss = 0.29394880
epoch: 70, loss = 0.27708119
epoch: 80, loss = 0.26287070
epoch: 90, loss = 0.25069037
epoch: 100, loss = 0.24010031
Accuracy = 91.2281%
